In [44]:
import numpy as np

In [54]:
def fit(X_train, Y_train):
    result = {}
    class_values = set(Y_train)
    for i in class_values:
        result[i] = {}
        result["total_data"] = len(Y_train)
        cur_data = (Y_train==i)
        X_cur = X_train[cur_data]
        Y_cur = Y_train[cur_data]
        num_features = X_train.shape[-1]
        result[i]["total_count"] = len(cur_data)
#         print("cur_data: " , cur_data.sum())
        for j in range(1,num_features+1):
            result[i][j] = {}
            all_values = set(X[:,j-1])
            for value in all_values:
                result[i][j][value] = (X_cur[:,j-1] == value).sum()
    return result

In [46]:
def label(col):
    l2 = col.mean()
    l1 = 0.5 * l2
    l3 = 1.5 * l2
    for i in range(len(col)):
        if col[i]<=l1:
            col[i]=0
        elif col[i]<=l2:
            col[i]=1
        elif col[i]<=l3:
            col[i]=2
        else:
            col[i]=3
    return col

In [47]:
def probability(dictionary,x,i):
    ans = np.log(dictionary[i]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[i].keys())-1
    for j in range(1,num_features+1):
        num = dictionary[i][j][x[j-1]] + 1
        deno = dictionary[i]["total_count"] + len(dictionary[i][j].keys())
        prob = np.log(num) - np.log(deno)
        ans += prob
    return ans

In [48]:
def predSinglePoint(dictionary, x):
    all_class = dictionary.keys()
    pmax = -1000
    bestclass = -1
    FirstRun = True
    for i in all_class:
        if i == "total_data":
            continue
        p = probability(dictionary, x, i)
        if(p>pmax or FirstRun):
            pmax = p
            bestclass = i
        FirstRun = False
    return bestclass

In [49]:
def predict(dictionary, X_test):
    ans = []
    for x in X_test:
        y = predSinglePoint(dictionary,x)
        ans.append(y)
    return ans

In [50]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target
for i in range(X.shape[-1]):
    X[:,i] = label(X[:,i])


In [58]:
from sklearn import model_selection

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X,Y, test_size = 0.25, random_state = 0)
dictionary = fit(X_train, Y_train)
# set(Y_train)
# dictionary.keys()

Y_pred = predict(dictionary, X_test)


In [60]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.94      1.00      0.97        16
          2       1.00      0.89      0.94         9

avg / total       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
